In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.2.0


In [3]:
#index
t = tf.constant([[1,2,3],[4,5,6]])
print(t)
print(t[:,1:])
print(t[...,1])

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor([2 5], shape=(2,), dtype=int32)


In [5]:
#ops
print(t+1)
print(tf.square(t))
print(t @ tf.transpose(t))

tf.Tensor(
[[2 3 4]
 [5 6 7]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 1  4  9]
 [16 25 36]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[14 32]
 [32 77]], shape=(2, 2), dtype=int32)


In [8]:
# numpy conversion
print(t.numpy())
print(np.square(t))
np_t = np.array([[1,1,1],[2,2,2]])
print(tf.constant(np_t))

[[1 2 3]
 [4 5 6]]
[[ 1  4  9]
 [16 25 36]]
tf.Tensor(
[[1 1 1]
 [2 2 2]], shape=(2, 3), dtype=int32)


In [11]:
#scaler
t = tf.constant(3.21213)
print(t.numpy())
print(t.shape)

3.21213
()


In [14]:
# string
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [15]:
# string array
t = tf.constant(['cafe', 'caffee', '咖啡'])
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor([b'cafe' b'caffee' b'\xe5\x92\x96\xe5\x95\xa1'], shape=(3,), dtype=string)
tf.Tensor([4 6 6], shape=(3,), dtype=int32)
tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 97, 102, 102, 101, 101], [21654, 21857]]>


In [18]:
# ragged tensor
r = tf.ragged.constant([[1], [1,2,3], [], [5]])
print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[1], [1, 2, 3], [], [5]]>
tf.Tensor([1 2 3], shape=(3,), dtype=int32)
<tf.RaggedTensor [[1, 2, 3]]>


In [21]:
# ops ragged tensor
r2 = tf.ragged.constant([[51, 52], [], [72]])
print(tf.concat([r, r2], axis=0))

<tf.RaggedTensor [[1], [1, 2, 3], [], [5], [51, 52], [], [72]]>


In [23]:
# ops ragged tensor
r3 = tf.ragged.constant([[2], [1,1,1], [22], [51]])
print(tf.concat([r, r3], axis=1))

<tf.RaggedTensor [[1, 2], [1, 2, 3, 1, 1, 1], [22], [5, 51]]>


In [24]:
print(r3.to_tensor())

tf.Tensor(
[[ 2  0  0]
 [ 1  1  1]
 [22  0  0]
 [51  0  0]], shape=(4, 3), dtype=int32)


In [28]:
# sparse tensor indices必须是排序的, 否则to_dense会报错
s = tf.SparseTensor(indices = [[0, 1], [2,3], [3, 3]], values=[1., 2., 3.], dense_shape=[4,4])
print(s)
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [2 3]
 [3 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]
 [0. 0. 0. 3.]], shape=(4, 4), dtype=float32)


In [30]:
# sparse tensor ops
s2 = s * 2.0
print(tf.sparse.to_dense(s2))

# sparse tensor not support + 
try:
    s3 = s + 1
except TypeError as ex:
    print(ex)


s4 = tf.constant([[1.,1.], [2., 2.], [3., 3.],[4., 4.]])
print(s.shape)
print(s4.shape)
print(tf.sparse.sparse_dense_matmul(s, s4))

#s = tf.sparse.reorder(s)

tf.Tensor(
[[0. 2. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 6.]], shape=(4, 4), dtype=float32)
unsupported operand type(s) for +: 'SparseTensor' and 'int'
(4, 4)
(4, 2)
tf.Tensor(
[[ 2.  2.]
 [ 0.  0.]
 [ 8.  8.]
 [12. 12.]], shape=(4, 2), dtype=float32)


In [21]:
# Variable
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [22]:
# assign
v.assign(v*2)
print(v.numpy())

v[0, 1].assign(42)
print(v.numpy())

v[1].assign([7., 8. ,9.])
print(v.numpy())

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 7.  8.  9.]]
